# Linear Programming Optimization With Python

Linear programming is a mathematical optimization technique used to optimize a linear objective function subject to a set of linear constraints. It involves finding the values of decision variables that minimize or maximize the objective function while satisfying the given constraints.

The objective function and constraints are represented as linear equations or inequalities, and the decision variables are the unknowns that need to be determined. The goal of linear programming is to find the optimal solution that satisfies all the constraints while optimizing the objective function.

Pulp, Pyomo, Scipy, OR-tools are among popular open-source modeling languages used for formulating and solving linear programming problems. They provide powerful tools for modeling and solving linear programming problems. They offer flexible and intuitive syntaxes for specifying optimization models and can be easily integrated into existing Python-based workflows. Additionally, they both support a wide range of solvers (glpk, cbc, gurobi, couenne and so on) making it easy to find the best solver for a particular problem.

This notebook contains 8 mini problems from various optimization topics. These problems are purposely designed to provide practice in formulating optimization problems and becoming familiar with popular optimization libraries in Python.

**1. Problem 1**

In [1]:
# Test available solvers that pulp can use in this machine
import pulp
pulp.listSolvers(onlyAvailable=True)

['GLPK_CMD']

**Using pyomo with glpk solver**

In [2]:
# a) Find manufacturing plan to optimize revenue

from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
for i in range(1,5):
    model.add_component(f"x{i}", Var(within=NonNegativeReals))

# Define the objective function
model.obj = Objective(expr=0.8*model.x1 + 0.3*model.x2 + 0.38*model.x3 + 0.4*model.x4, sense=maximize)

# Define the constraints
model.con1 = Constraint(expr=0.5*model.x1 + 0.2*model.x2 + 0.3*model.x3 + 0.6*model.x4 <= 600)
model.con2 = Constraint(expr=0.1*model.x1 + 0.4*model.x2 + 0.2*model.x3 + 0.5*model.x4 <= 800)

# Solve the linear programming problem
solver = SolverFactory('glpk', validate=False)
results = solver.solve(model)

# Print the solution
print(f"Optimal value: {model.obj()}")
print("Optimal solution: ")
for i in range(1,5):
    print(f"x{i} = {getattr(model,f'x{i}')()}")

Optimal value: 960.0
Optimal solution: 
x1 = 1200.0
x2 = 0.0
x3 = 0.0
x4 = 0.0


**Using pyomo with cbc solver**

In [3]:
# b) With new constraints and using cbc solver

# Create the model
model = ConcreteModel()

# Define the decision variables
for i in range(1,5):
    model.add_component(f"x{i}", Var(within=NonNegativeReals))

# Define the objective function
model.obj = Objective(expr=0.8*model.x1 + 0.3*model.x2 + 0.5*model.x3 + 0.4*model.x4, sense=maximize)

# Define the constraints
model.con1 = Constraint(expr=0.5*model.x1 + 0.2*model.x2 + 0.3*model.x3 + 0.6*model.x4 <= 600) # 600 kg
model.con2 = Constraint(expr=0.1*model.x1 + 0.4*model.x2 + 0.2*model.x3 + 0.5*model.x4 <= 800) # 800 kg
model.con3 = Constraint(expr=model.x1 + model.x2 >= 1000) 

# Solve the linear programming problem
solver = SolverFactory('cbc', executable='D:/PROJECTS/Data analytics/Solvers/bin/cbc.exe')
results = solver.solve(model)

# Print the solution
print(f"Optimal value: {model.obj()}")
print("Optimal solution: ")
for i in range(1,5):
    print(f"x{i} = {getattr(model,f'x{i}')()}")

Optimal value: 966.666665
Optimal solution: 
x1 = 1000.0
x2 = 0.0
x3 = 333.33333
x4 = 0.0


**2. Problem 2**

**Using pulp to solve**

In [4]:
import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_2", pulp.LpMaximize)

# Define the decision variables
x1, x2, x3, x4 = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(4)]

# Define the objective function
prob += 0.07 * x1 + 0.08 * x2 + 0.1 * x3 + 0.11 * x4

resource = 500 # unit in million

# Define the constraints
prob += x1 <= 100
prob += x2 <= 100
prob += x3 <= 150
prob += x4 <= 150
prob += x2 + x3 >= 0.5 * resource
prob += x1 >= 0.1 * resource

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(1,5):
    print(f"x{i} = {eval(f'x{i}.varValue')}")

Status: Optimal
Optimal value: 46.5
Optimal solution: 
x1 = 100.0
x2 = 100.0
x3 = 150.0
x4 = 150.0


**Using pyomo with gurobi solver**

In [5]:
from pyomo.environ import *

# Create the pulp model
model = ConcreteModel()

# Define the decision variables
for i in range(1,5):
    model.add_component(f"x{i}", Var(within=NonNegativeReals))

# Define the objective function
model.obj = Objective(expr=0.07*model.x1 + 0.08*model.x2 + 0.1*model.x3 + 0.11*model.x4, sense=maximize)

resource = 500 # unit in million

# Define the constraints
model.con1 = Constraint(expr=model.x1 <= 100) 
model.con2 = Constraint(expr=model.x2 <= 100) 
model.con3 = Constraint(expr=model.x3 <= 150) 
model.con4 = Constraint(expr=model.x4 <= 150) 
model.con5 = Constraint(expr=model.x1 >= 0.1 * resource) 
model.con6 = Constraint(expr=model.x2 + model.x3 >= 0.5 * resource) 

# Solve the linear programming problem using gurobi solver
solver = SolverFactory('gurobi', executable='D:/PROJECTS/Data analytics/Solvers/bin/gurobi/bin/gurobi.bat')
results = solver.solve(model)

# Print the solution
print(f"Optimal value: {model.obj()}")
print("Optimal solution: ")
for i in range(1,5):
    print(f"x{i} = {getattr(model,f'x{i}')()}")

Optimal value: 46.5
Optimal solution: 
x1 = 100.0
x2 = 100.0
x3 = 150.0
x4 = 150.0


**3. Problem 3**

**Using pulp to solve**

In [6]:
import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_3", pulp.LpMaximize)

# Define the decision variables
x1, x2, x3, x4 = [pulp.LpVariable(f"x{i}", lowBound=0, cat="Integer") for i in range(4)]

# Define the objective function
prob += 1000 * x1 + 1500 * x2 + 2500 * x3 + 3000 * x4

# Define the constraints
prob += x1 >= 5
prob += x2 >= 5
prob += x3 >= 5
prob += x4 >= 5
prob += x1 + x2 + x3 + x4 == 50
prob += 9 * x1 + 25 * x2 + 25 * x3 + 43 * x4 <= 1460

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value (USD): {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(1,5):
    print(f"x{i} = {eval(f'x{i}.varValue')}")

Status: Optimal
Optimal value (USD): 120500
Optimal solution: 
x1 = 5
x2 = 5
x3 = 24
x4 = 16


**4. Problem 4**

**Using pulp to solve**

In [7]:
import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_4", pulp.LpMaximize)

# Define the decision variables
# M=male, F=female, C=cut, M=mill
# 1 = MC, 2 = MM, 3 = FC, 4 = FM
x1, x2, x3, x4 = [pulp.LpVariable(f"x{i}", lowBound=0, cat="Integer") for i in range(4)]

# Define the objective function
prob += 30*x1 + 25*x2 + 28*x3 + 20*x4

# Define the constraints
prob += x1 + x2 == 32
prob += x3 + x4 == 20
prob += 30*x1 + 28*x3 == 25*x2 + 20*x4

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(1,5):
    print(f"x{i} = {eval(f'x{i}.varValue')}")

Status: Infeasible
Optimal value: 0
Optimal solution: 
x1 = 0
x2 = 0
x3 = 0
x4 = 0


The equality constraints are too difficult to meet, so there is no integer point that satisfies all the constraints as defined.

Representing x2 in terms of x1, x4 in terms of x3, and substituting into the third equation:

30x1 + 28x3 = 25*(32-x1) + 20*(20 - x3)
(30+25)x1 + (28+20)x3 = 2532 + 2020

If the variables are not required to be integers, then the system of constraint equations can be met.

However, when we require x1, x2, x3, and x4 to be integers, there is no feasible point. Geometrically, this means that the line (30+25)x1 + (28+20)x3 = 2532 + 2020 does not intersect any point where both x1 and x3 are integers.

**5. Problem 5**

**Using or-tools to solve**

In [8]:
from ortools.linear_solver import pywraplp

# Create the linear solver
solver = pywraplp.Solver.CreateSolver('GLOP')

# Define the decision variables
names = ['xA', 'xB', 'xC', 'xD']
variables = [solver.IntVar(0, solver.infinity(), name) for name in names]

# Define the objective function
coefficients = [250000, 350000, 380000, 850000]
obj = solver.Objective()

for var, coeff in zip(variables, coefficients):
    obj.SetCoefficient(var, coeff)
obj.SetMaximization()

# Define the constraints
constraints = [
    {'coefficients': [0.08, 0.12, 0.3, 0.21], 'upper_bound': 350},
    {'coefficients': [4, 9, 7, 12], 'upper_bound': 1000},
    {'coefficients': [1, 1, 3, 40], 'upper_bound': 2500}
]
for cons_data in constraints:
    constraint = solver.Constraint(-solver.infinity(), cons_data['upper_bound'])
    for var, coeff in zip(variables, cons_data['coefficients']):
        constraint.SetCoefficient(var, coeff)

# Solve the LP problem
solver.Solve()

# Get the optimal objective value
print('Optimal objective value:', solver.Objective().Value())

# Get the optimal decision variables
for variable in variables:
    print(variable.name() + ':', variable.solution_value())


Optimal objective value: 68581081.08108108
xA: 67.56756756756755
xB: 0.0
xC: 0.0
xD: 60.81081081081081


**6. Problem 6**

**Using pulp to solve**

In [9]:
# a) Problem as described
import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_6a", pulp.LpMinimize)

# Define the decision variables
factory = [5, 15, 20, 30]
warehouse6a = [10, 10, 10, 20, 20]
distance = [[5, 1, 4, 6, 7], [3, 4, 2, 7, 8], [4, 3, 1, 7, 9], [6, 5, 4, 9, 11]]

dim_i = len(factory)
dim_j = len(warehouse6a)

# Amount of goods in tons
x = [[pulp.LpVariable("x_{}_{}".format(i, j), lowBound=0, cat="Integer") for j in range(dim_j)] for i in range(dim_i)]

# Define the objective function as sum across rows and columns
prob += pulp.lpSum([[distance[i][j] * x[i][j] for i in range(dim_i)] for j in range(dim_j)])

# Define 4 constraints for 4 factories
for i in range(dim_i):
    prob += pulp.lpSum([x[i][j] for j in range(dim_j)]) == factory[i]
# and 5 contraints for warehouses
for j in range(dim_j):
    prob += pulp.lpSum([x[i][j] for i in range(dim_i)]) == warehouse6a[j]

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {x[i][j].varValue}")


Status: Optimal
Optimal value: 435
Optimal solution: 
x[0][0] = 0
x[0][1] = 5
x[0][2] = 0
x[0][3] = 0
x[0][4] = 0
x[1][0] = 10
x[1][1] = 0
x[1][2] = 0
x[1][3] = 0
x[1][4] = 5
x[2][0] = 0
x[2][1] = 0
x[2][2] = 10
x[2][3] = 0
x[2][4] = 10
x[3][0] = 0
x[3][1] = 5
x[3][2] = 0
x[3][3] = 20
x[3][4] = 5


**Using pyomo with couenne solver**

In [10]:
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
factory = [5, 15, 20, 30]
warehouse6a = [10, 10, 10, 20, 20]
distance = [[5, 1, 4, 6, 7], [3, 4, 2, 7, 8], [4, 3, 1, 7, 9], [6, 5, 4, 9, 11]]

dim_i = len(factory)
dim_j = len(warehouse6a)

# Amount of goods in tons
model.x = Var(range(dim_i), range(dim_j), within=NonNegativeIntegers)

# Define the objective function as sum across rows and columns
model.obj = Objective(expr=sum(distance[i][j] * model.x[i, j] for i in range(dim_i) for j in range(dim_j)), sense=minimize)

# Define 4 constraints for 4 factories
def factory_constraint(model, i):
    return sum(model.x[i, j] for j in range(dim_j)) == factory[i]
model.factory_constr = Constraint(range(dim_i), rule=factory_constraint)

# and 5 contraints for warehouses
def warehouse_constraint(model, j):
    return sum(model.x[i, j] for i in range(dim_i)) == warehouse6a[j]
model.warehouse_constr = Constraint(range(dim_j), rule=warehouse_constraint)

# Solve the LP problem using couenne solver
solver = SolverFactory('couenne', executable='D:/PROJECTS/Data analytics/Solvers/bin/couenne.exe')
results = solver.solve(model)

# Print the solution
print(f"Status: {results.solver.status}")
print(f"Optimal value: {model.obj.expr()}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {model.x[i, j].value}") 


Status: ok
Optimal value: 435.0
Optimal solution: 
x[0][0] = 0.0
x[0][1] = 3.0
x[0][2] = 0.0
x[0][3] = 0.0
x[0][4] = 2.0
x[1][0] = 6.0
x[1][1] = 0.0
x[1][2] = 0.0
x[1][3] = 0.0
x[1][4] = 9.0
x[2][0] = 2.0
x[2][1] = 3.0
x[2][2] = 10.0
x[2][3] = 5.0
x[2][4] = 0.0
x[3][0] = 2.0
x[3][1] = 4.0
x[3][2] = 0.0
x[3][3] = 15.0
x[3][4] = 9.0


In [11]:
# b) Probelem as described but with C3 = C4 = 15 tons
import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_6b", pulp.LpMinimize)

# Define the decision variables
factory = [5, 15, 20, 30]
warehouse6b = [10, 10, 15, 15, 20]
distance = [[5, 1, 4, 6, 7], [3, 4, 2, 7, 8], [4, 3, 1, 7, 9], [6, 5, 4, 9, 11]]

dim_i = len(factory)
dim_j = len(warehouse6b)

# Amount of goods in tons
x = [[pulp.LpVariable("x_{}_{}".format(i, j), lowBound=0, cat="Integer") for j in range(dim_j)] for i in range(dim_i)]

## Define the objective function
prob += pulp.lpSum([[distance[i][j] * x[i][j] for i in range(dim_i)] for j in range(dim_j)])

## Define the constraints
# Define 4 constraints for 4 factories
for i in range(dim_i):
    prob += pulp.lpSum([x[i][j] for j in range(dim_j)]) == factory[i]
# and 5 contraints for warehouses
for j in range(dim_j):
    prob += pulp.lpSum([x[i][j] for i in range(dim_i)]) == warehouse6b[j]
    
# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {x[i][j].varValue}")

Status: Optimal
Optimal value: 405
Optimal solution: 
x[0][0] = 0
x[0][1] = 5
x[0][2] = 0
x[0][3] = 0
x[0][4] = 0
x[1][0] = 10
x[1][1] = 0
x[1][2] = 0
x[1][3] = 0
x[1][4] = 5
x[2][0] = 0
x[2][1] = 0
x[2][2] = 15
x[2][3] = 0
x[2][4] = 5
x[3][0] = 0
x[3][1] = 5
x[3][2] = 0
x[3][3] = 15
x[3][4] = 10


**7. Problem 7**

**Using Scipy to solve**

In [12]:
# 7) Problem 7

import numpy as np
from scipy.optimize import linprog

# Define the objective function
obj_fun = [-10, -8, -7, -8, -9, -11]

# Define the constraints
# Worker: M=male, F=female
# Milling_machine: I, II, II

lhs_eq = [
    [1, 1, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 1],
    [1, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 1]
]
rhs_eq = [2, 3, 1, 2, 3]

# Define the bounds on the variables
bounds = [(0, None)] * 6

# Solve the LP problem
results = linprog(obj_fun, A_ub=lhs_eq, b_ub=rhs_eq, bounds=bounds, method="simplex")

# Print the solution
print(f"Status: {results.message}")
print(f"Optimal value: {-results.fun}")
roundRes = np.around(results.x)
print(f"Optimal solution: FI = {roundRes[0]}, FII = {roundRes[1]}, FIII = {roundRes[2]}, MI = {roundRes[3]}, MII = {roundRes[4]}, MIII = {roundRes[5]}")

Status: Optimization terminated successfully.
Optimal value: 51.0
Optimal solution: FI = 1.0, FII = 1.0, FIII = 0.0, MI = 0.0, MII = 0.0, MIII = 3.0


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
C:\Users\huyha\AppData\Local\Temp\ipykernel_11140\1732714596.py:26: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  results = linprog(obj_fun, A_ub=lhs_eq, b_ub=rhs_eq, bounds=bounds, method="simplex")


**8. Problem 8**

**Using pulp to solve**

In [13]:
# a) Solve the problem as described

import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_8a", pulp.LpMaximize)

# Define the decision variables
rice_type = [30, 20, 40]
field_type = [25, 25, 40]
productivity8a = [[4, 8, 6], [6, 9, 7], [5, 4, 6]] #tons/hectare

dim_i = len(rice_type)
dim_j = len(field_type)

# Productivity per hectares rice field
x = [[pulp.LpVariable("x_{}_{}".format(i, j), lowBound=0, cat="Integer") for j in range(dim_j)] for i in range(dim_i)]

# Define the objective function
prob += pulp.lpSum([[productivity8a[i][j] * x[i][j] for i in range(dim_i)] for j in range(dim_j)])

# Define the constraints
for i in range(dim_i):
    prob += pulp.lpSum([x[i][j] for j in range(dim_j)]) == rice_type[i]
    
for j in range(dim_j):
    prob += pulp.lpSum([x[i][j] for i in range(dim_i)]) == field_type[j]

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {x[i][j].varValue}")

Status: Optimal
Optimal value: 585
Optimal solution: 
x[0][0] = 0
x[0][1] = 25
x[0][2] = 5
x[1][0] = 0
x[1][1] = 0
x[1][2] = 20
x[2][0] = 25
x[2][1] = 0
x[2][2] = 15


In [14]:
# b) Formulate and solve the problem in the case where IR8 rice cannot be grown on type I field

import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_8b", pulp.LpMaximize)

# Define the decision variables
rice_type = [30, 20, 40]
field_type = [25, 25, 40]
productivity8b = [[0, 8, 6], [6, 9, 7], [5, 4, 6]] #tons/hectare

dim_i = len(rice_type)
dim_j = len(field_type)

# Productivity per hectares rice field
x = [[pulp.LpVariable("x_{}_{}".format(i, j), lowBound=0, cat="Integer") for j in range(dim_j)] for i in range(dim_i)]

# Define the objective function
prob += pulp.lpSum([[productivity8b[i][j] * x[i][j] for i in range(dim_i)] for j in range(dim_j)])


# Define the constraints
for i in range(dim_i):
    prob += pulp.lpSum([x[i][j] for j in range(dim_j)]) == rice_type[i]
    
for j in range(dim_j):
    prob += pulp.lpSum([x[i][j] for i in range(dim_i)]) == field_type[j]

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {x[i][j].varValue}")

Status: Optimal
Optimal value: 585
Optimal solution: 
x[0][0] = 0
x[0][1] = 25
x[0][2] = 5
x[1][0] = 0
x[1][1] = 0
x[1][2] = 20
x[2][0] = 25
x[2][1] = 0
x[2][2] = 15


The condition of IR8 rice cannot be grown on type I field does not affect the total harvest.